## Rapport Maël Olivier et Lina Kaddouri

## 3 / Least squares

Q 3.1/

Given that \(Aw = b\), we can express the components of \(Aw\) as:

(A w)_t = x(t)^{T} w_1 + w_0 - y(t) x(t)^{T} w_2 = b_t = y(t).


Therefore, we have:

y(t) = w_1^{T} x(t) + w_0 + w_2^{T} x(t) + 1.


Q 3.2/

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Loading data

data_matrix_train, COP_train, data_matrix_test, COP_test, names = np.load('data_center_data_matrix.npy', allow_pickle=True)

# Constructing matrices for min_w ||A w - b||_2**2

matrix_mean = np.mean(data_matrix_train, axis=0)
M = data_matrix_train - matrix_mean
matrix_std = np.std(M, axis=0)
M = M / matrix_std

A = np.hstack([M, np.ones((M.shape[0],1)), -(M.T * COP_train[:,3]).T])
b = COP_train[:,3]

w, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)

print("Solution (w):", w)


: 

Q 3.3/

In [ ]:
predictions_test = np.dot(A_test, w)
residuals_test = b_test - predictions_test
mse_test = np.mean(residuals_test**2)

print("Mean Squared Error on Test Set:", mse_test)

Q 3.4/

In [ ]:
data_matrix_train, COP_train, data_matrix_test, COP_test, names = np.load('data_center_data_matrix.npy', allow_pickle=True)

matrix_mean = np.mean(data_matrix_train, axis=0)
M = data_matrix_train - matrix_mean
matrix_std = np.std(M, axis=0)
M = M / matrix_std

lambda_value = 100
A_regularized = np.hstack([M, np.ones((M.shape[0], 1)), -(M.T * COP_train[:, 3]).T])
b_regularized = COP_train[:, 3]

lambda_matrix = lambda_value * np.eye(A_regularized.shape[1])

w_regularized, _, _, _ = np.linalg.lstsq(
    np.vstack([A_regularized, lambda_matrix]),
    np.concatenate([b_regularized, np.zeros(A_regularized.shape[1])]),
    rcond=None
)

print("Regularized Solution (w):", w_regularized)

M_test = (data_matrix_test - matrix_mean) / matrix_std
A_test_regularized = np.hstack([M_test, np.ones((M_test.shape[0], 1)), -(M_test.T * COP_test[:, 3]).T])
b_test_regularized = COP_test[:, 3]

predictions_test_regularized = np.dot(A_test_regularized, w_regularized)
residuals_test_regularized = b_test_regularized - predictions_test_regularized
mse_test_regularized = np.mean(residuals_test_regularized**2)

print("Mean Squared Error on Test Set with Regularization:", mse_test_regularized)



Q 3.5/

Le gradient est de la forme A^T( Aw - b) + λw


La dérivée seconde est de la forme (A^T)A + λI, qui est bien définie positive (si on multiplie d'un côté par X^T, et de l'autre par X, pour un X quelconque non nul de bonne dimension, on aura la somme de deux normes, ce qui est bien positif ou nul). f_1 est donc bien convexe.

Q 3.6/

In [ ]:
def gradient_f1(A, w, b, lam):
    return A.T @ (A @ w - b) + lam * w

def gradient_norm(grad):
    return np.linalg.norm(grad)

def gradient_descent(A, b, lam, alpha, max_iterations=1000, tolerance=1):
    w = np.zeros(A.shape[1])
    iterations = 0
    
    while iterations < max_iterations:
        grad = gradient_f1(A, w, b, lam)
        w = w - alpha * grad
        
        if gradient_norm(grad) <= tolerance:
            break
        
        iterations += 1
    
    return w, iterations

lam = 100                    
alpha = 0.1                  

w_optimal, num_iterations = gradient_descent(A, b, lam, alpha)
print("Optimal w:", w_optimal)
print("Number of iterations:", num_iterations)


## 4/ Regularization  for a sparse model

Q 4.1/

On peut prendre $f_{2}(w)=\frac{1}{2}\|A w-b\|_{2}^{2}$ $g_{2}(w)=\lambda\|w\|_{1}$

On a donc: $\nabla f_{2}(w)=A^{\top}(A w-b)$ et proxg2 $(v, r)=\operatorname{sign}(v) \times(|v|-t)_{+}$

Q 4.2/

In [ ]:
def proximal_operator_l1(v, lam, alpha):
    return np.sign(v) * np.maximum(np.abs(v) - alpha * lam, 0)

def proximal_gradient_method(A, b, lam, alpha, max_iterations=1000, tolerance=1e-6):
    w = np.zeros(A.shape[1])
    iterations = 0
    
    while iterations < max_iterations:
        gradient = A.T @ (A @ w - b)
        w_new = proximal_operator_l1(w - alpha * gradient, lam, alpha)
        
        if np.linalg.norm(w_new - w, ord=1) < tolerance:
            break
        
        w = w_new
        iterations += 1
    
    return w, iterations

lam = 200
alpha = 0.01

w_optimal, num_iterations = proximal_gradient_method(A, b, lam, alpha)
print("Optimal w:", w_optimal)
print("Number of iterations:", num_iterations)